In [1]:
import petpy
import pandas as pd
import numpy as np
import random
import os

In [2]:
petfinder=petpy.Petfinder(
    key='uTe0odJUWZhi3Lv0pyLteRLLJVv9Qv65GTH6zJDEBoEAlmd7kt', 
    secret='HGN1qte2FpD8TgaL9WtuZF9WDuPUq129RVxUYDR7')

In [11]:
# get the list of adoped DOGS for each of the states in the REMAINING regions (NOT the PNW,SOUTH CENTRAL, or NORTHEAST regions)
# time period: Jan 1, 2022 - June 30, 2022

# issues with nevada and hawaii

#remaining regions
#states = ['AK', 'AZ', 'CA', 'CO', 'DE', 'FL', 'GA', 'IA', 'IL', 'IN','KS', 'MD', 'MI', 'MN', 'MO', 'MT', 'NE', 'ND', 'OH', 'SC', 'SD', 'UT', 'VA', 'WI', 'WV', 'WY']

states = ["HI"]

for state in states:
    cats_state=petfinder.animals(animal_type='cat',
                  status='adopted',
                  before_date='2022-12-31', 
                  after_date='2022-01-01',
                  location=state,
                  pages=None
                  ,return_df=True)
    #print(dogs_state)
    file_loc = "data/other_regs/cats_"+state+"_jantodec2022.csv"
    print("file_loc, ", file_loc)
    cats_state.to_csv(file_loc)

file_loc,  data/other_regs/cats_HI_jantodec2022.csv


In [12]:
cats_hi = pd.read_csv("data/other_regs/cats_HI_jantodec2022.csv")

In [13]:
print(cats_hi.columns)

Index(['Unnamed: 0', 'id', 'organization_id', 'url', 'type', 'species', 'age',
       'gender', 'size', 'coat', 'tags', 'name', 'description',
       'organization_animal_id', 'photos', 'videos', 'status',
       'status_changed_at', 'published_at', 'distance', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 

In [14]:
print(cats_hi.shape)

(269, 51)


In [15]:
cats_hi.drop_duplicates(subset=['id'], inplace=True)

In [16]:
print("cats_hi_shape", cats_hi.shape)

cats_hi_shape (268, 51)


In [17]:
cats_hi['published_at']=pd.to_datetime(cats_hi['published_at'])
cats_hi['status_changed_at']=pd.to_datetime(cats_hi['status_changed_at'])

In [18]:
cats_hi['days_in_shelter']=cats_hi['status_changed_at']-cats_hi['published_at']
cats_hi['days_in_shelter']=cats_hi['days_in_shelter']/np.timedelta64(1,'D')
cats_hi['days_in_shelter']=cats_hi['days_in_shelter'].round()

In [19]:
cats_hi.columns

Index(['Unnamed: 0', 'id', 'organization_id', 'url', 'type', 'species', 'age',
       'gender', 'size', 'coat', 'tags', 'name', 'description',
       'organization_animal_id', 'photos', 'videos', 'status',
       'status_changed_at', 'published_at', 'distance', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 

In [20]:
cats_hi['breed'] = ''
# Extract primary and secondary breed from 'breeds' column
for i, row in cats_hi.iterrows():
    primary_breed = row['breeds.primary']
    secondary_breed = row['breeds.secondary']
    mixed = row['breeds.mixed']
    unknown_breed = row['breeds.unknown']
    
    if mixed:
        cats_hi.at[i, 'breed'] = 'Mixed'
    elif pd.notna(primary_breed):
        cats_hi.at[i, 'breed'] = primary_breed
    elif pd.notna(unknown_breed):
        cats_hi.at[i, 'breed'] = unknown_breed
    elif pd.notna(secondary_breed):
        cats_hi.at[i, 'breed'] = secondary_breed
    else:
        cats_hi.at[i, 'breed'] = 'Unknown'

In [21]:
cats_hi.shape

(268, 53)

In [22]:
# Select relevant columns
cats_hi = cats_hi[['id', 'type', 'species', 'age', 'gender', 'size', 'coat', 'tags', 'name', 'description', 'status', 
         'published_at', 'status_changed_at', 'days_in_shelter', 'breed',
         'colors.primary', 'attributes.spayed_neutered', 'attributes.house_trained', 'attributes.special_needs', 
         'attributes.shots_current', 'environment.dogs', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country']]

In [23]:
cats_hi.to_csv('data/other_regs/cats_HI_jantodec2022_cleaned', index=False)

In [26]:
df_hi = pd.read_csv("data/other_regs/cats_HI_jantodec2022_cleaned")
df_other = pd.read_csv("data/combined/us_minusNVHI_jantodec_adopted_dogsANDcats.csv")

/var/folders/js/_m5r698x29j4dvr0w18h6jdm0000gn/T/ipykernel_36778/2284017227.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_other = pd.read_csv("data/combined/us_minusNVHI_jantodec_adopted_dogsANDcats.csv")


In [27]:
print(df_hi.shape)
print(df_other.shape)

(268, 25)
(372834, 25)


In [28]:
us_all = pd.concat([df_hi, df_other])
print(us_all.shape)

(373102, 25)


In [ ]:
print(us_all['state'])

In [29]:
us_all.to_csv('data/combined/us_all_jantodec_adopted_dogsANDcats.csv', index=False)
# saved to google drive